- 가상환경: venv_chat_backend_test

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

print(f".env 파일경로: {find_dotenv()}")
load_dotenv()
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))

.env 파일경로: /home/a543979/hdegis-chat-backend/.env
/home/a543979/hdegis-chat-backend/key/pjt-dev-hdegis-app-454401-bd4fac2d452b.json


In [3]:
os.getenv("PROJECT_ID")

'pjt-dev-hdegis-app-454401'

In [4]:
import pandas as pd
import numpy as np

from google import genai
from google.genai import types

from utils import UserInputProcessor

In [5]:
genai_client = genai.Client(
    vertexai=True,
    project=os.getenv("PROJECT_ID"),
    location=os.getenv("LOCATION")
)

user_input_processor = UserInputProcessor()

In [8]:
df = pd.read_excel("QA 성능테스트 v4.xlsx")

/opt/miniforge3/envs/venv_chat_backend_test/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [6]:
HYDE_DOCUMENT_GENERATOR_PROMPT = '''
You are an AI assistant specialized in generating hypothetical documents based on user queries. Your task is to create a detailed, factual document that would likely contain the answer to the user's question. This hypothetical document will be used to enhance the retrieval process in a Retrieval-Augmented Generation (RAG) system.

Guidelines:
1. Carefully analyze the user's query to understand the topic and the type of information being sought.
2. Generate a hypothetical document that:
   a. Is directly relevant to the query
   b. Contains factual information that would answer the query
   c. Includes additional context and related information
   d. Uses a formal, informative tone similar to an encyclopedia or textbook entry
3. Structure the document with clear paragraphs, covering different aspects of the topic.
4. Include specific details, examples, or data points that would be relevant to the query.
5. Aim for a document length of 200-300 words.
6. Do not use citations or references, as this is a hypothetical document.
7. Avoid using phrases like "In this document" or "This text discusses" - write as if it's a real, standalone document.
8. Do not mention or refer to the original query in the generated document.
9. Ensure the content is factual and objective, avoiding opinions or speculative information.
10. Output only the generated document, without any additional explanations or meta-text.

User Question:
{user_query}

Generate a hypothetical document that would likely contain the answer to this query:
'''

In [10]:
def create_hyde(user_query):
    response = genai_client.models.generate_content(
        model='gemini-2.0-flash-001',
        contents=HYDE_DOCUMENT_GENERATOR_PROMPT.format(user_query=user_query)
    )
    return response.text

In [12]:
for i, row in df.iterrows():
        
    user_query = row['query']
    print(f"[{(i+1):02d}] {user_query}")
    
    user_query_en = user_input_processor.tranlate(user_query)

    hyde = create_hyde(user_query_en)
    print(f"ㄴ hyde: {hyde}\n")    
    break

[01] Are there any requirements regarding the operating method of the circuit breaker, such as spring-operated or hydraulic-operated?
ㄴ hyde: Circuit Breaker Operating Mechanisms

Circuit breakers, critical components in electrical power systems, employ various operating mechanisms to rapidly interrupt fault currents. These mechanisms provide the necessary force and speed to open and close the breaker contacts. Common operating methods include spring-operated, hydraulic-operated, pneumatic-operated, and direct manual operation. The selection of an operating mechanism depends on factors such as voltage level, interrupting capacity, operating speed requirements, and application (e.g., transmission substation, distribution system, industrial facility).

Spring-operated mechanisms utilize stored mechanical energy in compressed springs to drive the breaker contacts. These mechanisms are widely used for medium-voltage circuit breakers and are known for their reliability and relatively fast o

In [ ]:
prompt = """
You are a retrieval assistant that generates realistic hypothetical content from documents based on a user query and the type of documents in a folder.

You will receive:
- A folder description: describing the kinds of documents stored in the folder
- A user query: a question from the user

Your task is to:
1. Analyze the folder description to understand what kinds of technical or regulatory information these documents typically include.
2. Understand the user's query and what type of information would likely satisfy it.
3. Generate **3 to 5 short but informative sentences or paragraphs** that could realistically appear in one of those documents and would help answer the query.

Guidelines:
- Your outputs should reflect different possible scenarios or phrasing that such a document might include.
- Do not repeat or paraphrase the user query.
- Use precise, technical, and formal language similar to engineering specifications or regulatory documents.
- Each output should be standalone — as if it were extracted from a real document.
- Include conditions, preferences, limitations, or exceptions if relevant.
- Avoid chatty, explanatory, or meta language — focus on generating content that sounds like it comes from a real spec.

---

Example:

**Folder Description:**  
This folder contains standard specification documents from power companies and national authorities. These documents include technical requirements that must be followed during the design and manufacturing of high-voltage circuit breakers, such as insulation levels, operation methods, and environmental conditions.

**User Query:**  
Are there any requirements regarding the operating method of the circuit breaker, such as spring-operated or hydraulic-operated?

**Hypothetical Sentences or Paragraphs (3–5 diverse examples):**

1. The operating mechanism shall be of the motor-spring type. Any alternative mechanisms, such as hydraulic systems, require prior written approval from the client.

2. For circuit breakers installed in coastal environments, hydro-mechanical operating systems are preferred due to better sealing performance and environmental resilience.

3. Spring-operated mechanisms are not permitted for 400 kV class breakers, due to insufficient stored energy capacity under maximum fault conditions.

4. Both electromechanical and hydraulic mechanisms are allowed; however, the selected system must comply with the endurance requirements specified in section 6.2.

5. In projects involving REE, the accumulation of energy for circuit breaker operation must be mechanical (spring-based), regardless of the actuation system.

---

Now generate 3 to 5 realistic document-style sentences or short paragraphs that could answer the following query:

**Folder Description:**  
{folder_description}

**User Query:**  
{user_query}

**Hypothetical Document Snippets:**

"""

In [ ]:
folder_descriptions_expanded = {
    '1. International Standards': 'This folder includes international standards and specification documents essential for the design and testing of high-voltage circuit breakers. It contains technical references from global standardization bodies such as IEC and IEEE, and is used to support product development and the establishment of internal design standards.',
    'IEC': 'This folder contains standards and specification documents related to high-voltage equipment, issued by the International Electrotechnical Commission (IEC). It includes global guidelines for design, testing, and safety compliance.',
    'IEEE': 'This folder provides IEEE (Institute of Electrical and Electronics Engineers) standards, focusing on design specifications and testing procedures used primarily in North American markets.',
    '2. Type Test Reports': 'This folder contains type test reports of manufactured high-voltage circuit breakers. Reports are organized by model and test year, and include results on insulation performance, interruption capability, and other test metrics.',
    '145SP-3': 'Type test data and reports for the 145SP-3 circuit breaker model, covering various performance metrics.',
    '145 kV 40 kA MS (2017)': 'A test report for a 145kV / 40kA circuit breaker from the 145SP-3 model line, tested in 2017.',
    '300SR': 'Type test data and reports for the 300SR circuit breaker model.',
    '245 kV 50 kA MS (2020)': 'A test report for a 245kV / 50kA circuit breaker of the 300SR model, tested in 2020.',
    '245 kV 63 kA MS (2024)': 'A test report for a 245kV / 63kA circuit breaker of the 300SR model, tested in 2024.',
    '3. Customer Standard Specifications': 'This folder contains standard specification documents provided by various countries, power authorities, and individual consumers. These documents should be referred to when product designs need to reflect customer-specific requirements, such as local regulations or special installation or performance conditions.',
    'Endeavour Energy': 'Standard specification documents from Endeavour Energy, a power company based in Australia, which may include specific design requirements or operational preferences.',
    'OETC': 'Standard specification documents from OETC (Oman Electricity Transmission Company), detailing their technical and regulatory requirements.',
    'SEC': 'Standard specification documents from the Saudi Electricity Company (SEC), outlining design and compliance standards for circuit breakers.',
    'Iberdrola': 'Standard specification documents from Iberdrola, a Spanish power company, including customer-specific electrical design requirements.',
    'REE': 'Standard specification documents from Red Eléctrica de España (REE), a major transmission company in Spain, including their required standards and preferences for high-voltage circuit breakers.',
}

In [ ]:
folder_description = """
This folder contains standard specification documents provided by various countries, power authorities, and individual consumers. These documents should be referred to when product designs need to reflect customer-specific requirements, such as local regulations or special installation or performance conditions.
Standard specification documents from Red Eléctrica de España (REE), a major transmission company in Spain, including their required standards and preferences for high-voltage circuit breakers.
"""

In [ ]:
user_query = "Are there any requirements regarding the operating method of the circuit breaker, such as spring-operated or hydraulic-operated?"

In [ ]:
final_prompt = prompt.format(folder_description=folder_description.strip(), user_query=user_query).strip()

In [ ]:
print(final_prompt)

In [ ]:
response = genai_client.models.generate_content(
    model='gemini-2.0-flash-001',
    contents=final_prompt
)

In [ ]:
print(response.text)

In [ ]:
"""
You are an Intent-Aware Query Expansion Assistant for a retrieval-based Q&A system.

Your task is to expand user queries so they reflect the user's true intent more explicitly, in a way that is optimized for document retrieval.

You will receive:
- A folder description: a description of the types of documents the user wants to search in
- A user query: the initial question or request from the user

Follow these steps:
1. Read the folder description carefully and determine **what kinds of documents are available in that folder**, and **what types of information they are likely to contain**.
2. Analyze the user query to understand the **underlying purpose** of the question. What specific information is the user trying to find?
   - Think about implicit assumptions, goals, or examples the user may have in mind.
3. Based on steps 1 and 2, expand the user query into a more specific and fully-formed version. The expanded query should:
   - Clearly state what the user is trying to find
   - Be easy for a document retrieval system to match against available content
   - Include examples or clarification if necessary
   - Avoid simple paraphrasing

Return only the expanded query. Do not include explanations.

---

Example:

**Folder Description:**  
Standard specification documents provided by each country and customer. Refer to when delivering the product if you need a design that reflects customer requirements. This may include local application regulations and special requirements.

**User Query:**  
Are there any requirements regarding the operating method of the circuit breaker, such as spring-operated or hydraulic-operated?

**Expanded Query:**  
According to the standard specifications from the customer or country, are there any explicit requirements or preferences for the operating mechanism of the circuit breaker—for example, whether it must be spring-operated or hydraulic-operated? I am looking for information that specifies mandatory or preferred operation types that should be reflected in the product design.

---

Now expand the following user query using the given folder description:

**Folder Description:**  
{{folder_description}}

**User Query:**  
{{user_query}}

**Expanded Query:**

"""

## LangChain 

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

print(f".env 파일경로: {find_dotenv()}")
load_dotenv()
print(os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash-001", model_provider="google_vertexai")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

chain = prompt | model | StrOutputParser()

In [ ]:
chain.invoke({"topic": "bears"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

analysis_prompt = ChatPromptTemplate.from_template("is this a funny joke? {joke}")

composed_chain = {'joke': chain} | analysis_prompt | model | StrOutputParser()

In [ ]:
composed_chain.invoke({"topic": "bears"})

## utils

### 번역함수

In [ ]:
def translate_text(text):
    from google.cloud import translate_v2 as translate
    # Reference: https://cloud.google.com/translate/docs/basic/translate-text-basic?hl=ko#translate_translate_text-python

    translate_client = translate.Client()
    if isinstance(text, bytes):
        text = text.decode('utf-8')
    result = translate_client.translate(text, target_language='en')

    return result['translatedText']

In [ ]:
# 동작확인
text = 'IEC와 IEEE 규격에 따른 고압 차단기의 SF₆ 가스 연간 허용 누기율 기준은 어떻게 되나요?'
translate_text(text=text)

### 엑셀

In [ ]:
def revert_to_original_name(modified_name):
    """
    변환된 엑셀 표 이름을 원래대로 복원
    - 이중 언더스코어(`__`) → 슬래시(`/`)
    - 언더스코어(`_`) → 공백
    """
    # 슬래시 복원 (`__` → `/`)
    original_name = modified_name.replace("__", "/")
    # 언더스코어(`_`) → 공백
    original_name = original_name.replace("_", " ")

    return original_name

In [ ]:
def convert_to_excel_name(original_name):
    """
    엑셀 표 이름 규칙에 맞게 변환
    - 공백 → _
    - 슬래시(/) → __ (이중 언더스코어)
    - 숫자 + 점 제거 (예: '1. ' → '')
    - 특수 문자 제거 ('(', ')', '-')
    """
    import re
    # 숫자 + 점(`1.`) 제거
    modified_name = re.sub(r'^\d+\.\s*', '', original_name)
    # 슬래시(`/`) → `__`
    modified_name = modified_name.replace("/", "__")
    # 공백 → `_`
    modified_name = modified_name.replace(" ", "_")
    # 특수 문자 제거 (`(`, `)`, `-`)
    modified_name = re.sub(r'[()-]', '', modified_name)

    return modified_name

## 검색성능 평가를 위한 QA 데이터셋 가져오기

In [ ]:
import pandas as pd
df = pd.read_excel('QA 성능테스트 v4.xlsx')

In [ ]:
index_name_lst = ['hde_hvcb_text_004', 'hde_hvcb_text_0815', 'hde_hvcb_text_m_002']
emb_model_lst = ['text-embedding-004', 'text-embedding-005', 'text-multilingual-embedding-002']
k_lst = [3, 5, 10]

for iii in range(3):
    INDEX_NAME = index_name_lst[iii]
    EMB_MODEL_NAME = emb_model_lst[iii]
    print("=================================================")
    print(f"INDEX_NAME: {INDEX_NAME}")
    print(f"EMB_MODEL_NAME: {EMB_MODEL_NAME}")
    # es = Search()
    
    for K in k_lst:
        
        
        # 검색 성능 평가 (페이지 포함)
        hit_at_k = []
        reciprocal_ranks = []
        
        # 검색 결과 기록
        result_data = []
        
        for i, row in df.iterrows():
            ###################################### INPUT 변경 ######################################
            # input_content = []
            ###################################### INPUT 변경 ######################################
            
            # GT reference
            # gt_ref = []
            # for n in range(1, 14):
            #     if pd.isna(row[f"path_{n}"]):
            #         break
            #     path = row[f"path_{n}"]
            #     filename = row[f"filename_{n}"]
            #     page = int(row[f'page_{n}'])
            #     gt_ref.append((path, filename, page))
        
                ###################################### INPUT 변경 ######################################
                # content = row[f"content_{n}"]
                # input_content.append(content)
                ###################################### INPUT 변경 ######################################
                
            # Query
            # query = row['query']
            ###################################### INPUT 변경 ######################################
            # query += "\n".join(input_content)
            # query += row['gt_answer']
            ###################################### INPUT 변경 ######################################
            # lang = row['lang']
            # input_query = query
            
            # if EMB_MODEL_NAME in ['text-embedding-004', 'text-embedding-005']:
            #     multilinguality = False
            # elif EMB_MODEL_NAME in ['text-multilingual-embedding-002']:
            #     multilinguality = True
                
            # if (not multilinguality) and lang == 'kor':
                # input_query = translate_text(query)
            # print(query)
            # query_embedding = es.get_embedding(input_query)

            print(row['query'])
            # 적용할 필터
            user_defined_filters = row['폴더 선택 (검색 범위 지정)'].split('&')
            els_filters = [user_defined_filter.split('/')[-1].strip() for user_defined_filter in user_defined_filters]
            match_phrase_filters = [
                {'match_phrase': {'folder_levels': phrase}}
                for phrase in els_filters
            ]
            print(row['폴더 선택 (검색 범위 지정)'])
            print(user_defined_filters)
            print(match_phrase_filters)
            print("-------------------")
        
            # 검색 (w/ filter)
        #     resp = es.es.search(
        #         index=INDEX_NAME,
        #         knn={
        #             'field': 'embedding',
        #             'query_vector': query_embedding,
        #             'k': K,
        #             'num_candidates': 100,
        #             'filter': {
        #                 'bool': {
        #                     'should': match_phrase_filters,
        #                     'minimum_should_match': 1
        #                 }
        #             }
        #         },
        #         fields=["folder_levels", "pdf_name", "page"],
        #         source=False,
        #     )

            
        
        #     # 검색결과
        #     search_ref = []
        #     for idx, result in enumerate(resp['hits']['hits'], start=1):
        #         folder_levels = '/'.join(result['fields']['folder_levels'])
        #         folder_levels = convert_to_excel_name(folder_levels)
        #         pdf_name = result['fields']['pdf_name'][0]
        #         page = int(result['fields']['page'][0])
        #         score = result['_score']
        #         search_ref.append((folder_levels, pdf_name, page))
        
        #         # 결과 저장
        #         result_data.append({
        #             'filter': user_defined_filters,
        #             'query': query,
        #             'input_query': input_query,
        #             'search_rank': idx,
        #             'search_result': (folder_levels, pdf_name, page),
        #             'is_hit': (folder_levels, pdf_name, page) in gt_ref,
        #             'score': score,
        #             'gt_refs': gt_ref
        #         })
        
        #     # 검색성능 평가 (Hit@K)
        #     hit = any(item in gt_ref for item in search_ref) # True or False
        #     hit_at_k.append(hit)
        
        #     # 검색성능 평가 (Reciprocal Rank)
        #     reciprocal_rank = 0
        #     for rank, search_item in enumerate(search_ref, start=1):
        #         if search_item in gt_ref:
        #             reciprocal_rank = 1 / rank
        #             break
        #     reciprocal_ranks.append(reciprocal_rank)
        
        
        #     # 결과출력
        #     # print(f"[{i+1}]")
        #     # print(f"query: {query}")
        #     # print(f"filter: {row['폴더 선택 (검색 범위 지정)']}")
        #     # print(f"match_pharse_filters: {match_phrase_filters}")
        #     # print("----------------------------------")
        #     # print(f"search_ref:")
        #     # for s in search_ref:
        #     #     print(s)
        #     # print("----------------------------------")
        #     # print(f"gt_ref:")
        #     # for g in gt_ref:
        #     #     print(g)
        #     # print("----------------------------------")
        #     # print(f"Hit@K: {hit}")
        #     # print(f"rr: {reciprocal_rank}")
        #     # print("=======================================================================")
        
        
        # mrr = np.mean(reciprocal_ranks)
        # hit_at_k_score = np.mean(hit_at_k)
        # print(f"[최종결과]")
        # # print(f"EMB_MODEL_NAME: {EMB_MODEL_NAME}")
        # print(f"K: {K}")
        # print(f"Hit@{K}: {hit_at_k_score * 100:.2f}%")
        # print(f"Mean Reciporcal Rank (MRR): {mrr * 100:.2f}%")
        # print("-----")

## Elastic 초기화

In [ ]:
from elasticsearch import Elasticsearch
import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Elasitc Client 관련
ES_HOST = "https://node.hd-aic.com:30692"
ES_USER = "genai_hde"
ES_PASSWORD = "wpsAI1@"
ES_TIMEOUT = 3600
CA_CERT = None # CA_CERT는 필요하면 설정 (예: "../app/conf/ess-cloud.cer")

es = Elasticsearch(
    hosts=[ES_HOST],
    basic_auth=(ES_USER, ES_PASSWORD),
    verify_certs=False
)

## Elastic으로 검색

In [ ]:
INDEX_NAME = 'hde_hvcb_text_004'
EMBEDDING_MODEL = 'text-embedding-004'
TOP_K = 3

In [ ]:
resp = es.es.search(
                index=INDEX_NAME,
                knn={
                    'field': 'embedding',
                    'query_vector': query_embedding,
                    'k': K,
                    'num_candidates': 100,
                    'filter': {
                        'bool': {
                            'should': match_phrase_filters,
                            'minimum_should_match': 1
                        }
                    }
                },
                fields=["folder_levels", "pdf_name", "page"],
                source=False,
            )